# SCGNN Test - Kaza - No PCA

In [4]:
import pandas as pd
import sys
from pathlib import Path
sys.path.append('../..')
from src import config
from src.utils.dataset import TrafficDataset
from src.utils.model import TrafficPredictor
from torch.utils.data import DataLoader
import torch

In [2]:
DATASET_PATH = config.DATA_KAZA
jsonl_file = DATASET_PATH / "data.jsonl"
if jsonl_file.exists():
    df = pd.read_json(jsonl_file, lines=True)
    if "traffic_sequence" in df.columns:
        df = pd.DataFrame(df["traffic_sequence"].tolist())
else:
    df = pd.read_parquet(DATASET_PATH / "data.parquet")
df = df.fillna(0)
X_STEP, Y_STEP = 2, 1
data_array = df.values

In [5]:
# PCA kullanılmıyor
dataset = TrafficDataset(data_array, X_STEP, Y_STEP)
sensors = dataset.sensors
edges = []
for i in range(sensors - 1):
    edges.append([i, i + 1])
    edges.append([i + 1, i])
edge_index = torch.tensor(edges, dtype=torch.long).t()
predictor = TrafficPredictor('scgnn', sensors, X_STEP, Y_STEP, edge_index=edge_index, pca_model=dataset.pca if False else None)
predictor.load_model(config.PROJECT_ROOT.joinpath('models/scgnn_kaza_nopca.pt'))
loader = DataLoader(dataset, batch_size=10, num_workers=8)
preds = predictor.inference(loader)
display(pd.DataFrame(preds.squeeze()).head().iloc[:, :10])


,0,1,2,3,4,5,6,7,8,9
0,230.301056,273.384369,262.627441,256.179077,257.211151,258.377930,256.806824,254.517776,253.351135,252.767319
1,249.622986,244.774826,243.967529,256.897278,240.734665,274.272400,256.090790,239.522781,229.825974,246.795792
2,160.339798,123.978195,156.701584,128.019379,154.277435,135.293533,149.429871,118.724617,132.862686,122.765831
3,245.153366,215.247849,207.986099,167.171387,203.541504,173.231461,216.063797,193.027664,218.082077,201.915726
4,256.921173,262.577820,213.680908,208.023483,204.388107,216.509247,225.805740,232.271088,233.080170,243.585770
